<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-18 15:25:45
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  10.00 K
Current:  1.38 C
-------------------
Today PnL: -66.14 K (-0.48%)
Current PnL: -13.48 L (-9.39%)
CY Booked + Current PnL: -5.40 L (-3.76%)
-------------------
Total profit:  4.04 L
Total loss:  -17.52 L
-------------------
Total Booked + Current PnL: 21.58 L (18.52%)
Total Booked PnL: 35.06 L (30.08%)
Curr Year Booked PnL: 8.08 L (5.87%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 71.75 L (52.11%)
Deployed:  1.17 C
Current:  1.38 C
CAGR/XIRR %: 11.06%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-226.91,H-SC,9.74,91552.0,-3764.0,3799.0,-1.09,-3.95,4.15,0.04,161.0,-0.99,0.67,80.58,OX40N,NTT,MISC
70,SBIN,760.30,863.00,-9.61,M-LC,4.35,218095.0,16616.0,10599.0,-0.72,8.25,4.86,13.51,58.0,1.57,1.59,21.88,XY25,NTT,BANKS
86,UJJIVANSFB,52.77,53.00,51.69,M-SC,24.80,131787.0,-10692.0,11307.0,-0.57,-7.50,8.58,0.44,249.0,-0.95,0.96,55.54,OX40N,NTT,BANKS
17,BRITANNIA,5190.70,6446.05,15.22,H-LC,9.73,103374.0,9941.0,12653.0,-1.11,10.64,12.24,24.18,32.0,0.79,0.75,25.52,XY25,ATH,FMCG
39,HINDALCO,651.95,761.55,-21.55,H-LC,4.62,108152.0,3840.0,13692.0,0.21,3.68,12.66,16.81,9.0,0.28,0.79,20.32,X5K,ATH,METALS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,PGHH,13388.00,18172.17,-20.87,H-MC,7.81,199500.0,-1320.0,73077.0,-0.86,-0.66,36.63,35.73,88.0,-0.02,1.45,3.99,X40,ATH,FMCG
59,NATIONALUM,189.63,247.44,-50.15,H-MC,4.93,99735.0,-769.0,31407.0,-0.50,-0.77,31.49,30.49,82.0,-0.02,0.72,31.39,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,CIPLA,1492.70,1795.00,-22.84,H-LC,0.76,203157.0,-1343.0,42765.0,0.03,-0.66,21.05,20.25,11.0,-0.03,1.48,8.02,X40N,BTT,PHARMA
40,HINDUNILVR,2413.81,2723.00,-10.87,H-LC,1.31,279003.0,8656.0,25975.0,-0.75,3.20,9.31,12.81,16.0,0.33,2.03,16.68,XY25,NTT,FMCG
37,HCLTECH,1647.60,1957.01,6.70,H-LC,2.34,94458.0,-6046.0,24918.0,1.08,-6.02,26.38,18.78,10.0,-0.24,0.69,14.88,X40,ATH,IT
10,BAJAJHFL,122.26,152.00,-3.64,H-LC,2.88,199818.0,-688.0,49455.0,-0.64,-0.34,24.75,24.33,23.0,-0.01,1.45,15.23,X40N,BTT,FINANCE
41,HINDZINC,471.44,730.22,15.78,H-LC,3.18,189008.0,-16068.0,128639.0,-0.59,-7.84,68.06,54.89,3.0,-0.12,1.37,14.37,X5K,ATH,METALS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAMMAANCAP,170.35,326.0,-170.45,M-SC,5.72,81390.0,-20820.0,114206.0,3.60,-20.37,140.32,91.37,213.0,-0.18,0.59,32.12,XY25,NTT,FINANCE
94,WHIRLPOOL,1219.98,2270.0,-35.65,M-SC,11.60,108188.0,16690.0,62067.0,2.68,18.24,57.37,86.07,207.0,0.27,0.79,56.50,XR,NTT,DURABLES
95,WIPRO,243.46,420.0,-3.31,M-LC,5.83,165261.0,14316.0,95141.0,2.28,9.48,57.57,72.51,55.0,0.15,1.20,13.84,XR,NTT,IT
67,SAIL,130.64,228.0,96.05,M-MC,12.16,235208.0,10246.0,157401.0,2.24,4.55,66.92,74.53,193.0,0.07,1.71,36.56,XY24,BTT,STEEL
19,CAMPUS,294.86,393.0,-18.18,M-SC,1.44,170387.0,2612.0,53229.0,2.01,1.56,31.24,33.28,209.0,0.05,1.24,31.92,XY24,NTT,FOOTWEAR


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,ROUTE,1558.40,2249.26,-40.96,H-SC,16.19,63301.0,-39553.0,85153.0,-5.00,-38.46,134.52,44.33,117.0,-0.46,0.46,5.12,SR,ATH,IT
3,ALKYLAMINE,2347.98,4546.37,4.07,H-SC,20.89,96900.0,-4063.0,98596.0,-3.03,-4.02,101.75,93.63,147.0,-0.04,0.70,44.33,SR,ATH,CHEMICALS
90,VALIANTORG,512.64,838.00,-344.54,H-SC,19.75,104000.0,-29286.0,113880.0,-2.43,-21.97,109.50,63.47,153.0,-0.26,0.76,74.33,XR,NTT,CHEMICALS
83,TITAGARH,1097.23,1548.00,-1.23,H-SC,5.91,161954.0,-30061.0,108946.0,-2.12,-15.66,67.27,41.08,157.0,-0.28,1.18,37.07,XY24,NTT,ENGINEERING
18,BSOFT,628.85,836.99,-12.82,M-SC,12.90,73369.0,-36680.0,73105.0,-2.10,-33.33,99.64,33.10,212.0,-0.50,0.53,21.91,XR,ATH,IT


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-226.91,H-SC,9.74,91552.0,-3764.0,3799.0,-1.09,-3.95,4.15,0.04,161.0,-0.99,0.67,80.58,OX40N,NTT,MISC
86,UJJIVANSFB,52.77,53.0,51.69,M-SC,24.80,131787.0,-10692.0,11307.0,-0.57,-7.50,8.58,0.44,249.0,-0.95,0.96,55.54,OX40N,NTT,BANKS
78,SYMPHONY,1306.42,1306.0,-28.61,M-SC,12.31,152209.0,-18932.0,18874.0,-0.56,-11.06,12.40,-0.03,194.0,-1.00,1.11,10.15,OX40N,NTT,DURABLES
21,CERA,7989.07,8422.0,-12.89,H-SC,2.35,124488.0,-19315.0,27113.0,-1.94,-13.43,21.78,5.42,154.0,-0.71,0.90,37.38,OX40N,NTT,CERAMICS
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,9.74,148998.0,-25561.0,25598.0,-0.05,-14.64,17.18,0.02,129.0,-1.00,1.08,29.20,OX40N,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,VBL,492.64,672.85,-14.07,H-LC,14.58,260849.0,-4191.0,101131.0,-1.08,-1.58,38.77,36.58,1.0,-0.04,1.90,11.40,X40N,ATH,FMCG
32,ENRIN,1377.95,3176.70,NaN,NaN,0.86,75890.0,41441.0,3529.0,-0.61,120.30,4.65,130.54,2.0,11.74,0.55,16.05,AR,ATH,ELECTRICAL
41,HINDZINC,471.44,730.22,15.78,H-LC,3.18,189008.0,-16068.0,128639.0,-0.59,-7.84,68.06,54.89,3.0,-0.12,1.37,14.37,X5K,ATH,METALS
82,TCS,3794.03,4998.00,-23.52,H-LC,11.04,248656.0,-47278.0,141187.0,-0.66,-15.98,56.78,31.73,4.0,-0.33,1.81,0.00,X40,BTT,IT
48,INFY,1461.46,2275.00,-11.34,H-LC,7.30,284073.0,22472.0,123146.0,0.22,8.59,43.35,55.67,6.0,0.18,2.06,15.22,X40,BTT,IT


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,249.78,NaN,NaN,0.66,54334.0,13079.0,369.0,-0.67,31.70,0.68,32.59,197.0,35.44,0.39,27.92,XY24,ATH,IT
58,MEDANTA,1087.93,1486.00,19.36,H-SC,6.78,152969.0,26769.0,19412.0,-1.96,21.21,12.69,36.59,151.0,1.38,1.11,36.38,XY24,NTT,HEALTHCARE
32,ENRIN,1377.95,3176.70,NaN,NaN,0.86,75890.0,41441.0,3529.0,-0.61,120.30,4.65,130.54,2.0,11.74,0.55,16.05,AR,ATH,ELECTRICAL


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SAIL,130.64,228.00,96.05,M-MC,12.16,235208.0,10246.0,157401.0,2.24,4.55,66.92,74.53,193.0,0.07,1.71,36.56,XY24,BTT,STEEL
53,KPIGREEN,497.21,731.64,34.29,H-SC,10.50,137554.0,12257.0,46823.0,-1.00,9.78,34.04,47.15,144.0,0.26,1.00,72.09,MH,ATH,POWER
58,MEDANTA,1087.93,1486.00,19.36,H-SC,6.78,152969.0,26769.0,19412.0,-1.96,21.21,12.69,36.59,151.0,1.38,1.11,36.38,XY24,NTT,HEALTHCARE
20,CAMS,3643.00,5226.82,4.98,H-SC,2.88,117799.0,15795.0,28554.0,-0.77,15.48,24.24,43.48,121.0,0.55,0.86,34.86,X40N,ATH,MISC
15,BLUESTARCO,1646.70,2326.38,-1.75,H-SC,12.03,184260.0,19590.0,48387.0,-1.65,11.90,26.26,41.28,119.0,0.40,1.34,20.83,X40N,ATH,AC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,0.86,75890.0,41441.0,3529.0,-0.61,120.30,4.65,130.54,2.0,11.74,0.55,16.05,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1219.98,2270.00,-35.65,M-SC,11.60,108188.0,16690.0,62067.0,2.68,18.24,57.37,86.07,207.0,0.27,0.79,56.50,XR,NTT,DURABLES
53,KPIGREEN,497.21,731.64,34.29,H-SC,10.50,137554.0,12257.0,46823.0,-1.00,9.78,34.04,47.15,144.0,0.26,1.00,72.09,MH,ATH,POWER
57,MASFIN,326.60,399.50,-10.57,H-SC,14.89,101685.0,3705.0,18161.0,-1.31,3.78,17.86,22.32,148.0,0.20,0.74,47.48,XR,ATH,FINANCE
95,WIPRO,243.46,420.00,-3.31,M-LC,5.83,165261.0,14316.0,95141.0,2.28,9.48,57.57,72.51,55.0,0.15,1.20,13.84,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,1.33,19171.0,-30422.0,1622.0,-0.56,-61.34,8.46,-58.07,201.0,-18.76,0.14,28.57,XY24,ATH,MISC
28,DIGITIDE,188.38,249.78,NaN,NaN,0.66,54334.0,13079.0,369.0,-0.67,31.70,0.68,32.59,197.0,35.44,0.39,27.92,XY24,ATH,IT
74,SIS,477.00,477.00,2307.25,M-SC,4.54,64770.0,-16320.0,16322.0,-0.46,-20.13,25.20,0.00,236.0,-1.00,0.47,31.27,OX40N,NTT,MISC
24,COFFEEDAY,59.14,80.00,-49.70,L-SC,21.17,75725.0,-37824.0,77876.0,-1.55,-33.31,102.84,35.27,266.0,-0.49,0.55,85.34,XR,NTT,HOTELS
92,VIPIND,488.80,489.00,-226.91,H-SC,9.74,91552.0,-3764.0,3799.0,-1.09,-3.95,4.15,0.04,161.0,-0.99,0.67,80.58,OX40N,NTT,MISC


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,RELAXO,902.64,1176.00,-28.05,H-SC,17.81,65018.0,-50520.0,85512.0,-0.97,-43.73,131.52,30.28,132.0,-0.59,0.47,27.02,X40N,NTT,FOOTWEAR
16,BLUSPRING,226.45,94.95,NaN,NaN,1.33,19171.0,-30422.0,1622.0,-0.56,-61.34,8.46,-58.07,201.0,-18.76,0.14,28.57,XY24,ATH,MISC
3,ALKYLAMINE,2347.98,4546.37,4.07,H-SC,20.89,96900.0,-4063.0,98596.0,-3.03,-4.02,101.75,93.63,147.0,-0.04,0.70,44.33,SR,ATH,CHEMICALS
24,COFFEEDAY,59.14,80.00,-49.70,L-SC,21.17,75725.0,-37824.0,77876.0,-1.55,-33.31,102.84,35.27,266.0,-0.49,0.55,85.34,XR,NTT,HOTELS
57,MASFIN,326.60,399.50,-10.57,H-SC,14.89,101685.0,3705.0,18161.0,-1.31,3.78,17.86,22.32,148.0,0.20,0.74,47.48,XR,ATH,FINANCE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.48
1,25,40.92
2,50,69.95


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.04
LC    34.88
MC    22.03
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.83
X40N     14.34
X40      12.44
XY25     10.97
XR        9.40
OX40N     8.48
AR        8.19
X5K       2.16
MH        1.72
X200      1.34
SR        1.16
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.03
H-SC    25.60
H-MC    18.91
M-SC    14.95
M-LC     5.80
M-MC     2.76
L-SC     1.49
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.16,-0.46,34.17
IT,11.76,-12.26,67.40
FINANCE,8.52,-11.09,51.40
BANKS,7.79,-6.57,50.06
MISC,5.95,-17.60,72.91
PAINTS,5.58,-13.74,36.15
ELECTRICAL,5.32,-0.69,48.40
HEALTHCARE,4.38,-0.20,26.81
AUTO,4.33,-13.72,63.73


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2609352.0,25
AR,1024550.0,10
XR,1000160.0,13
X40,744914.0,10
X40N,579235.0,12
XY25,430791.0,8
OX40N,305665.0,11
SR,183749.0,2
X5K,142331.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2302239.0,26
M-SC,1572280.0,20
H-LC,1329813.0,21
H-MC,1110194.0,15
M-MC,298723.0,2
M-LC,264027.0,4
L-SC,193443.0,3
L-MC,53055.0,1
L-LC,45568.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,897848.0,7
M-SC,XY24,774581.0,7
H-SC,AR,514993.0,3
H-LC,X40,477839.0,6
H-MC,XY24,428583.0,4
H-SC,XR,329394.0,4
H-LC,X40N,301851.0,6
M-MC,XY24,298723.0,2
M-SC,XR,268699.0,4


In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 37.0 seconds
